# Equity Linked Notes - Reverse Convertible (basket)

This notebook demonstrates how to access and use the functionalities of **Reverse Convertible Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Reverse Convertible structure with basket underlying.

**You will be able to:**
* Define Reverse Convertible instrument with basic parameters
* Configure pricing parameters
* Evaluate Reverse Convertible Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Reverse Convertible, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Reverse_Convertible_basket",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Basket", type = "string", value="TTEF_PA|IBE_MC|ENEI_MI"),
        sp.NameTypeValue(name="FirstAsset", type = "string", value="Perf_TTEF_PA"),
        sp.NameTypeValue(name="LastAsset", type = "string", value="Perf_ENEI_MI"),
        sp.NameTypeValue(name="BasketFunction", type = "string", value="WorstOf"),
        sp.NameTypeValue(name="BasketPerf", type = "string", value="If(\"BasketFunction\" == \"WorstOf\", Min(FirstAsset[t]:LastAsset[t]), IF(\"BasketFunction\" == \"Average\", Average(FirstAsset[t]:LastAsset[t],True), 0))"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="09/07/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="23/07/2029"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="09/07/2029"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["09/07/2026", "09/07/2026", "23/07/2026", "23/07/2026"],
                    ["09/07/2027", "09/07/2027", "23/07/2027", "23/07/2027"],
                    ["10/07/2028", "10/07/2028", "24/07/2028", "24/07/2028"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="30%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="5.15%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Repeat(Basket,#)",
            "Repeat(Basket,Perf_#)",
            "Performance",
            "Coupon",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(#)",
            "",
            "",
            "",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(#)",
            "",
            "",
            "Receive CouponRate",
            "",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(#)",
            "#[t]/#[StrikeDate]",
            "BasketPerf",
            "Receive (MaturityDate, CouponRate)",
            "Receive (MaturityDate, 100%)",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

rev_convert_basket = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

rev_convert_basket_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 9, 10),  # Set your desired valuation date
    report_ccy="EUR",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "TTEF.PA",
                  underlying_name = "TTEF_PA",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "IBE.MC",
                  underlying_name = "IBE_MC",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "ENEI.MI",
                  underlying_name = "ENEI_MI",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

try:
    response = sp.price(
        definitions=[rev_convert_basket],
        pricing_preferences=rev_convert_basket_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )

    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Reverse_Convertible_basket",
    "dealCcy": "EUR",
    "reportCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Coupon": 0.196453891,
        "ENEI_MI": 27.58237854,
        "IBE_MC": 56.160257697,
        "OptionAtMaturity": -0.128786247,
        "PERF_ENEI_MI": 0.761364559,
        "PERF_IBE_MC": 0.82801378,
        "PERF_TTEF_PA": 0.756099155,
        "Performance": 0.49338251,
        "Price": 990.344463888,
        "PriceIn%": 99.034446389,
        "Settlement": 0.92267682,
        "TTEF_PA": 179.526638193
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_rev_convert_basket_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_rev_convert_basket_valuation)

,Field,Value
0,marketValueInDealCcy,990.344464
1,marketValueInReportCcy,990.344464


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=rev_convert_basket_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,TTEF_PA,IBE_MC,ENEI_MI,PERF_TTEF_PA,PERF_IBE_MC,PERF_ENEI_MI,Performance,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-07-09,1.0000,53.280000,15.655000,8.119000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2026-07-09,0.9839,50.375439,15.320217,7.728538,0.000000,0.000000,0.000000,0.000000,0.051463,0.000000,0.000000,0.000000,EUR,Future
2,2027-07-09,0.9653,48.046354,14.925689,7.384651,0.000000,0.000000,0.000000,0.000000,0.051461,0.000000,0.000000,0.000000,EUR,Future
3,2028-07-10,0.9450,45.817522,14.514501,7.056359,0.000000,0.000000,0.000000,0.000000,0.051456,0.000000,0.000000,0.000000,EUR,Future
4,2029-07-09,0.9235,43.620912,14.035969,6.693403,0.818711,0.896581,0.824412,0.534239,0.051453,0.999083,-0.139451,1072.353689,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_df_rev_convert_basket_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_df_rev_convert_basket_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,5.094380
1,deltaAmountInReportCcy,5.094380
2,gammaAmountInDealCcy,2.368480
3,gammaAmountInReportCcy,2.368480
4,vegaAmountInDealCcy,-14.629700
5,vegaAmountInReportCcy,-14.629700
6,thetaAmountInDealCcy,0.523689
7,thetaAmountInReportCcy,0.523689
